In [2]:
%load_ext autoreload
%autoreload 2
import dataloader
import tomotopy as tp
from itertools import chain
import tqdm
import pandas as pd
import numpy as np
import slda
import post_classifier
import bow

In [29]:
POSTPATH = './Data/crowd/train/shared_task_posts.csv'
LABELPATH = './Data/crowd/train/crowd_train.csv'
USERPATH = './Data/crowd/train/task_C_train.posts.csv'

users = dataloader.load_user_subset_from_train(USERPATH, subset = 1000)
    
user_to_post, post_to_words, post_to_metadata = dataloader.load_posts(POSTPATH, user_subset = users, append_title = True)
post_to_label = dataloader.load_classification(LABELPATH, user_to_post, post_to_words, post_to_metadata)
filtered_data, sw_posts, sw_timestamps = dataloader.filter_posts(post_to_label, post_to_metadata)
print(len(filtered_data))
filtered_data = dataloader.filter_near_SW(filtered_data,post_to_metadata, sw_timestamps)
print(len(filtered_data))

filtered_data = dataloader.filter_stopwords(filtered_data)
sw_posts = dataloader.filter_stopwords(sw_posts)


FOLDERPATH = './Processing/crowd_processed/'
dataloader.save_to_folder(FOLDERPATH, user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps)


  0%|          | 5813/2038753 [00:00<00:34, 58125.34it/s]


Filtering subset...


 45%|████▍     | 910635/2038753 [00:13<00:16, 69271.19it/s]


 87%|████████▋ | 1775524/2038753 [00:26<00:04, 64142.29it/s]


100%|██████████| 2038753/2038753 [00:30<00:00, 67373.25it/s]

  1%|          | 485/57015 [00:00<00:11, 4840.64it/s]

...


 97%|█████████▋| 55294/57015 [00:20<00:00, 4604.66it/s]


100%|██████████| 57015/57015 [00:20<00:00, 2731.27it/s]

  0%|          | 0/55255 [00:00<?, ?it/s]

55255
Filtering posts far away from SW posts...



100%|██████████| 55255/55255 [00:00<00:00, 165944.03it/s]


29200


In [30]:
FOLDERPATH = './Processing/crowd_processed/'
user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps = dataloader.load_from_folder(FOLDERPATH)

In [31]:
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'a']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'b']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'c']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'd']))
print(len(filtered_data))

878
203
632
2025
29200


In [172]:
model = slda.train_slda_model_from_data(filtered_data, topics=40)

100%|██████████| 29200/29200 [00:00<00:00, 79810.60it/s]


Beginning sLDA training...
Iteration: 0	Log-likelihood: -10.112761424227246
Iteration: 100	Log-likelihood: -9.165663327710861
Iteration: 200	Log-likelihood: -9.051985439234114
Iteration: 300	Log-likelihood: -9.011560400621805
Iteration: 400	Log-likelihood: -8.98686379426409
Iteration: 500	Log-likelihood: -8.974676971519218
Iteration: 600	Log-likelihood: -8.96527641842238
Iteration: 700	Log-likelihood: -8.96427435423438
Iteration: 800	Log-likelihood: -8.959241327797196
Iteration: 900	Log-likelihood: -8.957747114654223
Finished Training


In [173]:
slda_coefficients = model.get_regression_coef(0)
data = []
for k in range(model.k):
    top_words = model.get_topic_words(k, top_n=40)
    words = [word for (word, float) in top_words]
    words = ", ".join(words)
    data.append([words, slda_coefficients[k]])
    
indices = np.array(slda_coefficients).argsort()
data = np.array(data)
data = data[indices]

pd.DataFrame(data, columns=["Topic", "Suicidality Coefficient"])

,Topic,Suicidality Coefficient
0,"person, babes, big, girl, f, sexy, hair, teen,...",-5.338107109069824
1,"live, mail, order, person, webcam, sexy, seeds...",-5.166828155517578
2,"url, us, min, person, survey, gt95, short, stu...",-4.828995704650879
3,"person, video, music, song, porn, one, show, r...",-4.37593936920166
4,"person, vs, persons, 1, 2, 3, character, chara...",-4.2679266929626465
5,"person, team, year, round, league, pick, first...",-4.244767665863037
6,"person, app, design, new, iphone, apple, mobil...",-4.2146711349487305
7,"person, bike, new, car, area, ride, night, roc...",-4.1955742835998535
8,"map, person, pvp, online, training, server, ma...",-4.1137237548828125
9,"’, ”, “, poker, play, hands, plo, rough, sex, ...",-3.8254079818725586


In [174]:
print(data[39][0])

im, dont, like, get, know, want, feel, life, ive, really, cant, people, even, never, going, much, need, time, things, go, friends, think, help, one, way, still, always, every, years, someone, good, bad, tell, getting, love, fucking, doesnt, anything, say, back


In [175]:
vector = slda.get_topic_vecs(model, filtered_data)

  0%|          | 3/29200 [00:00<17:25, 27.94it/s]

Getting topic distributions...


100%|██████████| 29200/29200 [14:39<00:00, 33.20it/s]


In [176]:
#Uncomment to print example feature vector
#print(vector['hw4uh'][0])

In [183]:
X = np.array([ vector[key][0] for key in vector.keys()])
y = np.array([ vector[key][1] for key in vector.keys()])
y= y.reshape(np.shape(y)[0])

In [184]:
print(np.shape(X))
print(np.shape(y))

(29200, 40)
(29200,)


In [185]:
p_clf = post_classifier.PostClassification("RbfSVM")
p_clf.train(X,y)

In [186]:
y_pred = p_clf.test(X)

In [187]:
p_clf.get_metrics(y, y_pred)

{'accuracy': 0.7223287671232876,
 'precision': 0.17404351087771944,
 'recall': 0.8019753086419753,
 'f1': 0.286016202888341}

In [188]:
print(sum(y_pred))
print(sum(y))

9331
2025


In [1]:
word2index,index2word = bow.generate_vocabulary(filtered_data)
vector2 = bow.get_vectors_from_post_set(filtered_data, word2index)

NameError: name 'bow' is not defined

In [203]:
print(word2index)

({'<OOV>': 0, 'reddit': 1, 'twitter': 2, 'stop': 3, 'accounts': 4, 'made': 5, 'specifically': 6, 'content': 7, 'spam': 8, 'subreddit': 9, 'lot': 10, 'coverage': 11, 'going': 12, 'game': 13, 'basically': 14, 'saying': 15, 'thing': 16, 'varying': 17, 'degrees': 18, 'quality': 19, 'snuff': 20, 'community': 21, 'im': 22, 'sure': 23, 'scouring': 24, 'every': 25, 'site': 26, 'know': 27, 'order': 28, 'find': 29, 'scraps': 30, 'information': 31, 'want': 32, 'post': 33, 'built': 34, 'upvoting': 35, 'spamming': 36, 'could': 37, 'easily': 38, 'linked': 39, 'directly': 40, 'either': 41, 'swtorcom': 42, 'website': 43, 'person': 44, 'page': 45, 'money': 46, 'grab': 47, 'taking': 48, 'subredditcan': 49, 'encouraging': 50, 'dae': 51, 'make': 52, 'crazy': 53, 'movements': 54, 'alone': 55, 'would': 56, 'never': 57, 'people': 58, 'watching': 59, 'simply': 60, 'one': 61, 'ever': 62, 'jack': 63, 'motions': 64, 'flail': 65, 'arms': 66, 'dance': 67, 'weirdly': 68, 'unless': 69, 'tell': 70, 'someone': 71, 'el

In [198]:
print(vector2)

{'hw4uh': ([58, 0], [0]), 'i87j9': ([34, 0], [0]), '2tqjks': ([7, 0], [0]), '2ude6n': ([19, 0], [0]), '2ufui5': ([81, 0], [0]), '2vqdhe': ([13, 0], [0]), '2vztle': ([62, 0], [0]), '2w83s8': ([18, 0], [0]), 'd6rnd': ([2, 0], [0]), 'd6xue': ([6, 0], [0]), 'd9qvi': ([2, 0], [0]), 'dbtzv': ([14, 0], [0]), 't5iw6': ([37, 0], [0]), 't7jrf': ([2, 0], [0]), 'tc19h': ([40, 0], [0]), 'tfof4': ([2, 0], [0]), 'tfol5': ([2, 0], [0]), 'toiq9': ([9, 0], [0]), 'txutn': ([23, 0], [0]), 'u1rvg': ([1, 0], [0]), 'gexy7': ([3, 0], [0]), 'nhfdn': ([7, 0], [0]), 'nhfem': ([5, 0], [0]), 'nhpy8': ([4, 0], [0]), 'nilpl': ([8, 0], [0]), 'nk9py': ([15, 0], [0]), 'nlyhr': ([9, 0], [0]), 'nsbku': ([7, 0], [0]), 'nwznd': ([1, 0], [0]), 'o05i9': ([26, 0], [0]), 'o172t': ([17, 0], [0]), 'o1def': ([11, 0], [0]), 'o1q5b': ([14, 0], [0]), 'o24he': ([18, 0], [0]), 'o3h5m': ([118, 0], [0]), 'o4liu': ([20, 0], [0]), 'o5lzu': ([1, 0], [0]), 'o5wyi': ([2, 0], [0]), 'o826v': ([13, 0], [0]), 'o9fxw': ([3, 0], [0]), 'oa2i2': ([1